### 📐 LLL Algorithm: Wikipedia Implementation & Tests

This implementation of the Lenstra–Lenstra–Lovász (LLL) lattice basis reduction algorithm is based on the description from [Wikipedia](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm).

The algorithm has been implemented in Python and extended with custom test cases to verify correctness, lattice preservation, and reduced vector lengths across various dimensions.

In [14]:
import numpy as np


def gram_schmidt(basis):
    n = len(basis)
    dim = len(basis[0])

    ortho = [np.zeros(dim) for _ in range(n)]
    mu = np.zeros((n, n))

    for i in range(n):
        b_i = basis[i]
        projection = np.zeros(dim)

        for j in range(i):
            mu[i, j] = np.dot(b_i, ortho[j]) / np.dot(ortho[j], ortho[j])
            projection += mu[i, j] * ortho[j]

        ortho[i] = b_i - projection

    return ortho, mu



def lll_reduce(basis, delta=0.75, verbose=False):

    basis = [b.copy() for b in basis]
    n = len(basis)
    k = 1


    while k < n:
        ortho, mu = gram_schmidt(basis)

        #print(basis)
        for j in range(k - 1, -1, -1):  # j < k
            if abs(mu[k, j]) > 0.5:
                r = round(mu[k, j])
                basis[k] -= r * basis[j]
                ortho, mu = gram_schmidt(basis)

        norm_sq_prev = np.dot(ortho[k - 1], ortho[k - 1])
        norm_sq_curr = np.dot(ortho[k], ortho[k])

        lhs = delta * norm_sq_prev
        rhs = norm_sq_curr + mu[k, k - 1]**2 * norm_sq_prev

        if lhs > rhs:
            basis[k], basis[k - 1] = basis[k - 1], basis[k].copy()

            k = max(k - 1, 1)
        else:
            k += 1

    return basis





In [15]:
import import_ipynb
import basis_reduction_2d as br2d; # type: ignore ## static funcs error?

def bases_equal_2d(basis1, basis2):
    def normalize(v):
        v = np.array(v)
        return v if v[0] > 0 or (v[0] == 0 and v[1] >= 0) else -v

    def sorted_basis(u, v):
        return sorted([normalize(u).tolist(), normalize(v).tolist()])

    return sorted_basis(basis1[0], basis1[1]) == sorted_basis(basis2[0], basis2[1])

def tests_brlll(basis_list, verbose=False):

    tests_amount = len(basis_list)
    tests_passed = 0

    results = []

    for i, basis in enumerate(basis_list):
        reduced = lll_reduce(basis)

        same = br2d.check_same_lattice(*basis, *reduced)

        original_len = min(np.linalg.norm(v) for v in basis)
        reduced_len = min(np.linalg.norm(v) for v in reduced)
        improved = reduced_len <= original_len

        result = int(same and improved)
        if result:
            tests_passed += 1

        if verbose:
            print(f"{'✅' if result else '❌'} Test {i + 1}: {'PASSED' if result else 'FAILED'}")
            print("Initial basis:")
            for v in basis:
                print(" ", v)
            print("Reduced basis:")
            for v in reduced:
                print(" ", v)
            print()

        results.append({
            "basis": [v.tolist() for v in reduced],
            "result": result
        })

    if verbose:
        print(f"\n📊 {tests_passed}/{tests_amount} tests passed.")

    return results



### ✅ LLL Reduction Tests (2D)

We test whether different 2D basis reduction methods (e.g. LLL, classical) produce equivalent bases.

Bases are normalized (up to sign and order) before comparison.

Matches confirm correct lattice reduction.

In [16]:
basis_list = br2d.generate_n_2d_bases(10)
test_results_br2d = br2d.tests_br2d(basis_list)
test_results_brlll = tests_brlll(basis_list)

for i in range(len(test_results_br2d)):
    res_2d = test_results_br2d[i]
    res_lll = test_results_brlll[i]

    b1_2d, b2_2d = res_2d["b1"], res_2d["b2"]
    b1_lll, b2_lll = res_lll["basis"]
    match = bases_equal_2d([b1_2d,b2_2d], [b1_lll, b2_lll])

    if match:
        print(f"Test {i+1}: ✅ MATCH")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")
    else:
        print(f"Test {i+1}: ❌ DIFFERENT")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")




Test 1: ✅ MATCH
  🔹 br2d  → b1 = [22 14], b2 = [ 21 -28]
  🔸 brlll → b1 = [22, 14], b2 = [21, -28]
Test 2: ✅ MATCH
  🔹 br2d  → b1 = [6 6], b2 = [ 15 -13]
  🔸 brlll → b1 = [-6, -6], b2 = [15, -13]
Test 3: ✅ MATCH
  🔹 br2d  → b1 = [-17 -44], b2 = [-36  37]
  🔸 brlll → b1 = [-17, -44], b2 = [-36, 37]
Test 4: ✅ MATCH
  🔹 br2d  → b1 = [-12   4], b2 = [ -7 -28]
  🔸 brlll → b1 = [-12, 4], b2 = [7, 28]
Test 5: ✅ MATCH
  🔹 br2d  → b1 = [-12 -46], b2 = [-46  24]
  🔸 brlll → b1 = [-46, 24], b2 = [-12, -46]
Test 6: ✅ MATCH
  🔹 br2d  → b1 = [-21 -34], b2 = [-47  33]
  🔸 brlll → b1 = [-21, -34], b2 = [-47, 33]
Test 7: ❌ DIFFERENT
  🔹 br2d  → b1 = [-32   0], b2 = [-16  47]
  🔸 brlll → b1 = [32, 0], b2 = [16, 47]
Test 8: ✅ MATCH
  🔹 br2d  → b1 = [ -7 -39], b2 = [-37  21]
  🔸 brlll → b1 = [-37, 21], b2 = [-7, -39]
Test 9: ✅ MATCH
  🔹 br2d  → b1 = [-15  -2], b2 = [-3 28]
  🔸 brlll → b1 = [15, 2], b2 = [-3, 28]
Test 10: ✅ MATCH
  🔹 br2d  → b1 = [-22   3], b2 = [  2 -35]
  🔸 brlll → b1 = [-22, 3], b2 = [-

In [17]:
basis = [
    np.array([1, -1, 3]),
    np.array([1,  0, 5]),
    np.array([1,  2, 6])
]

test_results_br2d = lll_reduce(basis)
print(test_results_br2d)



[array([ 1, -1,  0]), array([-1,  0,  1]), array([1, 1, 1])]
